In [3]:
import stable_baselines3
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from gym.wrappers import GrayScaleObservation
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from nes_py.wrappers import JoypadSpace
import os

In [4]:
# 保存最优模型
monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir,exist_ok=True)
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env,keep_dim=True)
env = Monitor(env, monitor_dir)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4,channels_order='last')

In [5]:
learning_rate = 1e-6
n_steps = 128
tensorboard_log = r'./tensorboard_log/'
model = PPO("CnnPolicy", env, verbose=1,
        learning_rate=learning_rate,
        n_steps=n_steps,tensorboard_log=tensorboard_log
)

Using cuda device
Wrapping the env in a VecTransposeImage.


[Auto saving best model](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/4_callbacks_hyperparameter_tuning.ipynb#scrollTo=adsKMvDkRUn0)

In [6]:
import os

import numpy as np

from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.results_plotter import load_results, ts2xy
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq, log_dir, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            print('self.n_calls: ', self.n_calls)
            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print("Num timesteps: {}".format(self.num_timesteps))
                    print(
                        "Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(
                            self.best_mean_reward, mean_reward
                        )
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print("Saving new best model at {} timesteps".format(x[-1]))
                        print("Saving new best model to {}.zip".format(self.save_path))
                    self.model.save(self.save_path)

        return True

In [8]:
log_dir = monitor_dir 
callback1 = SaveOnBestTrainingRewardCallback(check_freq=10, log_dir=log_dir)

model.learn(total_timesteps=150,callback=callback1)

Logging to ./tensorboard_log/PPO_2
self.n_calls:  10
self.n_calls:  20
self.n_calls:  30
self.n_calls:  40
self.n_calls:  50
self.n_calls:  60
self.n_calls:  70
self.n_calls:  80
self.n_calls:  90
self.n_calls:  100
self.n_calls:  110
self.n_calls:  120
----------------------------
| time/              |     |
|    fps             | 127 |
|    iterations      | 1   |
|    time_elapsed    | 1   |
|    total_timesteps | 128 |
----------------------------
self.n_calls:  130
self.n_calls:  140
self.n_calls:  150
self.n_calls:  160
self.n_calls:  170
self.n_calls:  180
self.n_calls:  190
self.n_calls:  200
self.n_calls:  210
self.n_calls:  220
self.n_calls:  230
self.n_calls:  240
self.n_calls:  250
-------------------------------------------
| time/                   |               |
|    fps                  | 84            |
|    iterations           | 2             |
|    time_elapsed         | 3             |
|    total_timesteps      | 256           |
| train/                  |     